# DaftAcademy: Python 4 Beginners

## Wykład 6: CZYSZCZENIE I WIZUALIZACJA DANYCH

### 4 XII 2018

### Paweł M. Święcki

-----------------------------------

# 0. KONTEKST

## Format danych

Plik CSV: `personal_data.csv`:

Nagłówek (pierwszy wiersz):

```
first_name,last_name,id_number,employment_start_date,monthly_salary,department,multisport
```
Dane (pozostałe wiersze):
```
Bernard,Adamski,LO/45418/2016,2016-07-24,5220.00,LOGISTYKA,True
Ryszard,Zakrzewski,KA/70437/2014,2014-01-06,6525.00,KADRY,False
Weronika,Mazur,ZA-42348-2017,2017-06-19,22200.00,ZARZĄD,True
```

### Poszczególne kolumny

### `first_name`

Imię pracownika.

### `last_name`

Nazwisko pracownika.

### `id_number`
ID pracownika. 

Składa się on z: **`{dwuliterowy_skrót_nazwy_działu}/{numer}/{rok_zatrudnienia}`**.

Np. `KA/70437/2014`, `LO/45418/2016`, `MA/82480/2016`, itp.

### `employment_start_date`
Data zadrudnienia pracownika w formacie `YYYY-MM-DD`.

### `monthly_salary`
Miesięczne wynagrodzenie pracownika.

### `department`
Dział, w którym dana osoba pracuje.

Jeden z: `{'ZARZĄD', 'IT', 'KSIĘGOWOŚĆ', 'KADRY', 'LOGISTYKA', 'SPRZEDAŻ', 'MARKETING', 'CZYSTOŚĆ', 'OCHRONA'}`.

### `multisport`

Czy osoba ma kartę multisport? `'True'` albo `'False'`.

## Zadania do wykonania


1. Wyznaczyć średnią i medianę wynagrodzeń.

2. Wyznaczyć średnią i medianę wynagrodzeń w poszczególnych działach firmy.

3. Sprawdzić, czy posiadanie karty multisport związane jest z wysokością zarobków.

4. Określić liczbę osób pracujących w poszczególnych działach firmy.

5. Znaleźć osobę zarabiającą najwięcej oraz osobę zarabiającą najmniej.

6. Sprawdzić kiedy zatrudniony został pierwszy pracownik działu kadr.


Podobne zadania będziemy wykonywać często. Warto więc przygotować sobie do tego narzędzia, by za bardzo się nie napracować :D

# 1. ŁADOWANIE I CZYSZCZENIE DANYCH

## Ładowanie danych

In [73]:
import csv

in_filename = 'personal_data.csv'
out_filename = 'clean_personal_data.csv'

# with open(in_filename) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for line in csv_reader:
#         print(line)

Szybki Google pokazuje, że znak `0x...` jest literą `...` z kodowania `Windows-1250`. Python spodziewa się `UTF-8`.

Google ...

Dokumentacja: https://docs.python.org/3/howto/unicode.html.

In [74]:
with open(in_filename, encoding='Windows-1250') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for line in list(csv_reader)[:10]:
        print(line)

['first_name', 'last_name', 'id_number', 'employment_start_date', 'monthly_salary', 'department', 'multisport']
['Urszula', 'Szewczyk', 'CZ/594959/2014', '2014-09-06T00:00:00', '2436.9', 'CZYSTOŚĆ', 'false']
['Stanisława', 'Kowalska', 'LO/374979/2010', '2010-04-20', '3407.4', 'LOGISTYKA', '0']
['Ilona', 'Pawlak', 'SP/598919/2014', '2014-04-09', '4676.760000000001', 'SPRZEDAŻ', 'true']
['Marek', 'Kwiatkowski', 'MA/913484/2010', '2010-09-17  ', '5533.968000000002', 'MARKETING', 'ma']
['Barbara ', 'Wiśniewska', 'SP/441969/2017', '2017-09-10T00:00:00', '4 731,210000000001', 'SPRZEDAŻ', 'tak']
['Robert', 'Walczak', 'SP/996950/2012', '2012-05-09', '5306.895000000001', 'SPRZEDAŻ', '1']
['LUCJAN', 'Kaźmierczak', 'OC/778427/2018', '2018-03-29', '3746.6000000000004 ', 'OCHRONA', 'Nie']
['Beata', 'Pawlak', 'KS/892760/2017', '2017-12-19', '5184.0', 'KSIĘGOWOŚĆ', 'NIE MA']
['HENRYK', 'Rutkowski', 'SP/111937/2012  ', '2012-11-25', '5 055,299999999999 ', 'SPRZEDAŻ', 'false']


In [75]:
def get_csv_lines():
    with open(in_filename, encoding='Windows-1250') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            yield row  # <== `yield`, czyli to nie funkcja, a generator

In [76]:
list(get_csv_lines())[:5]

[['first_name',
  'last_name',
  'id_number',
  'employment_start_date',
  'monthly_salary',
  'department',
  'multisport'],
 ['Urszula',
  'Szewczyk',
  'CZ/594959/2014',
  '2014-09-06T00:00:00',
  '2436.9',
  'CZYSTOŚĆ',
  'false'],
 ['Stanisława',
  'Kowalska',
  'LO/374979/2010',
  '2010-04-20',
  '3407.4',
  'LOGISTYKA',
  '0'],
 ['Ilona',
  'Pawlak',
  'SP/598919/2014',
  '2014-04-09',
  '4676.760000000001',
  'SPRZEDAŻ',
  'true'],
 ['Marek',
  'Kwiatkowski',
  'MA/913484/2010',
  '2010-09-17  ',
  '5533.968000000002',
  'MARKETING',
  'ma']]

Główna zaleta generatorów to małe zużycie pamięci. Bardzo istotne w operowaniu na wielkich plikach.

Nazwy pól w pierwszym wierszu, dane w kolejnych. Niewygodne, warto razem to mieć.

**`DictReader`** to the rescue!

Zob. https://docs.python.org/3/library/csv.html#csv.DictReader.

In [77]:
def get_csv_lines():
    with open(in_filename, encoding='Windows-1250') as csv_file:
        reader = csv.DictReader(csv_file, delimiter=',')
        for row in reader:
            yield row

In [78]:
list(get_csv_lines())[:3]

[OrderedDict([('first_name', 'Urszula'),
              ('last_name', 'Szewczyk'),
              ('id_number', 'CZ/594959/2014'),
              ('employment_start_date', '2014-09-06T00:00:00'),
              ('monthly_salary', '2436.9'),
              ('department', 'CZYSTOŚĆ'),
              ('multisport', 'false')]),
 OrderedDict([('first_name', 'Stanisława'),
              ('last_name', 'Kowalska'),
              ('id_number', 'LO/374979/2010'),
              ('employment_start_date', '2010-04-20'),
              ('monthly_salary', '3407.4'),
              ('department', 'LOGISTYKA'),
              ('multisport', '0')]),
 OrderedDict([('first_name', 'Ilona'),
              ('last_name', 'Pawlak'),
              ('id_number', 'SP/598919/2014'),
              ('employment_start_date', '2014-04-09'),
              ('monthly_salary', '4676.760000000001'),
              ('department', 'SPRZEDAŻ'),
              ('multisport', 'true')])]

Zamiast zwykłego dicta mamy **`OrderedDict`**. Zob. https://docs.python.org/3.7/library/collections.html#collections.OrderedDict.

In [79]:
first_person = list(get_csv_lines())[0]

print(first_person['first_name'])
print(first_person['employment_start_date'])

Urszula
2014-09-06T00:00:00


Bardzo wygodne, dużo lepsze, niż:

```python
first_person[3]  # które to pole?
```

## Czyszczenie danych

### Ogólny przegląd danych i wstępne czyszczenie

Zobaczmy dane w kolumnie `first_name`...

In [80]:
people = get_csv_lines()
unique_names = {
    person['first_name']
    for person in people
}
sorted(unique_names)[:10]

['',
 ' ',
 '  ',
 'ADAM   ',
 'ALEKSANDER',
 'ALEKSANDRA',
 'ALFRED',
 'ALFRED ',
 'ALFREDA',
 'ALICJA']

**`sorted`** zwraca listę posortowanych elementów z przekazanego iterable: https://docs.python.org/3/library/functions.html#sorted.

**`.sort`** jest metodą listy i sotruje ją in place: https://docs.python.org/3/tutorial/datastructures.html#more-on-lists.

Mamy przynajmniej dwa problemy:

- Różny zapis pod względem małych i wielkich liter.

- Zbędne spacje (ang. whitespace, co jest szerszym terminem, zob. https://en.wikipedia.org/wiki/Whitespace_character).

Zacznijmy od usunięcia zbędnych whitespace...

In [81]:
from collections import OrderedDict


def clean_personal_data(lines):
    for line in lines:
        # cleaning common to all fields
        clean_line = clean_all_items(line)
    
        # TODO: specialized cleaning
        
        yield clean_line
        
def clean_all_items(row):
    clean_row = OrderedDict()
    for key, value in row.items():
        clean_value = value.strip()        # <== usuwanie wiszących spacji
        clean_value = clean_value or None  # <== zamiana na None 
        clean_row[key] = clean_value       #       w przypadku pustego stringa
    
    return clean_row

Co to takiego to `x = coś or None`? Konsola...

Zob. https://www.geeksforgeeks.org/short-circuiting-techniques-python/.

Metoda **`strip`**: https://docs.python.org/3/library/stdtypes.html#str.strip.

Przepuścmy nasze dane przez `clean_personal_data`:

In [82]:
lines = get_csv_lines()
people = clean_personal_data(lines)
unique_names = {
    person['first_name']
    for person in people
    if person['first_name'] is not None
}
sorted(unique_names)[:25]

['ADAM',
 'ALEKSANDER',
 'ALEKSANDRA',
 'ALFRED',
 'ALFREDA',
 'ALICJA',
 'ALINA',
 'ALOJZY',
 'ANETA',
 'ANIELA',
 'ANNA',
 'ANTONI',
 'ARKADIUSZ',
 'ARTUR',
 'Adam',
 'Adela',
 'Agata',
 'Agnieszka',
 'Aldona',
 'Aleksander',
 'Aleksandra',
 'Alfred',
 'Alfreda',
 'Alicja',
 'Alina']

Czyli niepotrzebne whitespace usunięte, ale wciąż trzeba ogarnąć wielkie/małe litery...

Kod zaczyna się komplikować, więc najpierw:

1. Wrzućmy wszystko do klasy.

2. Dodajmy (na razie puste) metody czyszczące poszczególne typy pól.

In [83]:
from collections import OrderedDict

class PersonalDataCleaner:
    """
    Cleaner of personal data. 
    
    Usage:
      Pass an iterable of dicts to `clean` method, which 
      will yield dicts with cleaned data.
    """

    def clean(self, rows):
        for row in rows:
            # cleaning common to all fields
            clean_row = self._clean_all_items(row)

            # specialized cleaning
            clean_row['first_name'] = (
                self._clean_name(clean_row['first_name'])
            )
            clean_row['last_name'] = (
                self._clean_name(clean_row['last_name'])
            )
            clean_row['id_number'] = (
                self._clean_id_number(clean_row['id_number'])
            )
            clean_row['employment_start_date'] = (
                self._clean_date(clean_row['employment_start_date'])
            )
            clean_row['monthly_salary'] = (
                self._clean_monetary_value(clean_row['monthly_salary'])
            )
            clean_row['department'] = (
                self._clean_department(clean_row['department'])
            )
            clean_row['multisport'] = (
                self._clean_multisport(clean_row['multisport'])
            )
    
            yield clean_row
            
    def _clean_all_items(self, row):
        clean_row = OrderedDict()
        for key, value in row.items():
            clean_value = value.strip()
            clean_value = clean_value or None
            clean_row[key] = clean_value
        return clean_row
        

    def _clean_name(self, name): return name

    def _clean_id_number(self, id_number): return id_number

    def _clean_date(self, date): return date

    def _clean_monetary_value(self, monetary_amount): return monetary_amount

    def _clean_department(self, department): return department

    def _clean_multisport(self, multisport): return multisport


Działa?

In [84]:
lines = get_csv_lines()
cleaner = PersonalDataCleaner()
people = cleaner.clean(lines)

In [85]:
list(people)[:5]

[OrderedDict([('first_name', 'Urszula'),
              ('last_name', 'Szewczyk'),
              ('id_number', 'CZ/594959/2014'),
              ('employment_start_date', '2014-09-06T00:00:00'),
              ('monthly_salary', '2436.9'),
              ('department', 'CZYSTOŚĆ'),
              ('multisport', 'false')]),
 OrderedDict([('first_name', 'Stanisława'),
              ('last_name', 'Kowalska'),
              ('id_number', 'LO/374979/2010'),
              ('employment_start_date', '2010-04-20'),
              ('monthly_salary', '3407.4'),
              ('department', 'LOGISTYKA'),
              ('multisport', '0')]),
 OrderedDict([('first_name', 'Ilona'),
              ('last_name', 'Pawlak'),
              ('id_number', 'SP/598919/2014'),
              ('employment_start_date', '2014-04-09'),
              ('monthly_salary', '4676.760000000001'),
              ('department', 'SPRZEDAŻ'),
              ('multisport', 'true')]),
 OrderedDict([('first_name', 'Marek'),
             

Działa!

### Czyszczenie `first_name` i `last_name`

Wróćmy do czyszczenia wielkich/małych liter...

In [86]:
def clean_name(name):
    if name is None:
        return None
    return name.title()

Testy!

In [87]:
assert clean_name('ARKADIUSZ') == 'Arkadiusz'
assert clean_name('Emilia') == 'Emilia'
assert clean_name('katarzyna') == 'Katarzyna'
assert clean_name(None) == None

Przepuśćmy wszystkie imiona przez `clean_name`:

In [88]:
people = PersonalDataCleaner().clean(get_csv_lines())

unique_names = {
    clean_name(person['first_name'])
    for person in people
}
unique_names

{'Adam',
 'Adela',
 'Agata',
 'Agnieszka',
 'Aldona',
 'Aleksander',
 'Aleksandra',
 'Alfred',
 'Alfreda',
 'Alicja',
 'Alina',
 'Alojzy',
 'Andrzej',
 'Aneta',
 'Aniela',
 'Anna',
 'Antoni',
 'Antonina',
 'Arkadiusz',
 'Artur',
 'Barbara',
 'Beata',
 'Bernadeta',
 'Bernard',
 'Bogdan',
 'Bogumiła',
 'Bogusław',
 'Bogusława',
 'Bolesław',
 'Bożena',
 'Bronisław',
 'Bronisława',
 'Cecylia',
 'Celina',
 'Cezary',
 'Czesław',
 'Czesława',
 'Daniela',
 'Danuta',
 'Dariusz',
 'Dorota',
 'Edmund',
 'Edward',
 'Edyta',
 'Eleonora',
 'Elżbieta',
 'Emilia',
 'Eugenia',
 'Eugeniusz',
 'Ewa',
 'Feliks',
 'Franciszek',
 'Franciszka',
 'Gabriela',
 'Genowefa',
 'Gertruda',
 'Grażyna',
 'Grzegorz',
 'Halina',
 'Hanna',
 'Helena',
 'Henryk',
 'Henryka',
 'Honorata',
 'Ilona',
 'Irena',
 'Ireneusz',
 'Iwona',
 'Izabela',
 'Jacek',
 'Jadwiga',
 'Jan',
 'Janina',
 'Janusz',
 'Jarosław',
 'Jerzy',
 'Joanna',
 'Jolanta',
 'Julia',
 'Julian',
 'Justyna',
 'Józef',
 'Józefa',
 'Karol',
 'Karolina',
 'Katarz

Nice!

Potem zamienimy funkcję `clean_name` na metodę `_clean_name` w klasie...

### Czyszczenie `multisport`

In [89]:
people = PersonalDataCleaner().clean(get_csv_lines())

{person['multisport'] for person in people}

{'0',
 '1',
 'FAŁSZ',
 'False',
 'Fałsz',
 'MA',
 'NIE',
 'NIE MA',
 'Nie',
 None,
 'PRAWDA',
 'Prawda',
 'TAK',
 'Tak',
 'True',
 'false',
 'ma',
 'nie',
 'nie ma',
 'tak',
 'true'}

Co to jest?!

Zróbmy funkcję mapującą string na odpowiednią wartość.

In [90]:
STRINGS_REPRESENTING_TRUE = {'1', 'prawda', 'tak', 'ma', 'true'}
STRINGS_REPRESENTING_FALSE = {'0', 'fałsz', 'nie', 'nie ma', 'false'}

map_string_to_true = {
    k: True
    for k in STRINGS_REPRESENTING_TRUE
}
map_string_to_false = {
    k: False
    for k in STRINGS_REPRESENTING_FALSE
}

map_string_to_bool = {**map_string_to_true, **map_string_to_false}


def map_string_to_boolean(something):
    if something is None:
        return None
    something = something.lower()
    return map_string_to_bool[something]

In [91]:
map_string_to_bool

{'ma': True,
 'true': True,
 'prawda': True,
 '1': True,
 'tak': True,
 '0': False,
 'nie': False,
 'false': False,
 'fałsz': False,
 'nie ma': False}

Dopiszmy testy!

In [92]:
assert map_string_to_boolean('1') == True
assert map_string_to_boolean('0') == False
assert map_string_to_boolean('prawDa') == True
assert map_string_to_boolean('faŁsZ') == False
assert map_string_to_boolean(None) == None

Zobaczmy jaki zbiór wyników otrzymamy przepuszczając dane "produkcyjne" przez nią.

In [93]:
people = PersonalDataCleaner().clean(get_csv_lines())

{map_string_to_boolean(person['multisport']) for person in people}

{False, None, True}

Logika trójwartościowa.

### Czyszczenie `department`

In [94]:
people = PersonalDataCleaner().clean(get_csv_lines())

{person['department'] for person in people}

{'CZYSTOŚĆ',
 'IT',
 'KADRY',
 'KSIĘGOWOŚĆ',
 'LOGISTYKA',
 'MARKETING',
 None,
 'OCHRONA',
 'SPRZEDAŻ',
 'ZARZĄD'}

Wszystko wygląda ok, aż dziwne! Nie zmieniamy więc cleaner'a `_clean_department`.

### Czyszczenie `id_number`

In [95]:
people = PersonalDataCleaner().clean(get_csv_lines())

id_numbers = [person['id_number'] for person in people]

In [96]:
id_numbers[:10]

['CZ/594959/2014',
 'LO/374979/2010',
 'SP/598919/2014',
 'MA/913484/2010',
 'SP/441969/2017',
 'SP/996950/2012',
 'OC/778427/2018',
 'KS/892760/2017',
 'SP/111937/2012',
 'LO/530128/2009']

In [97]:
id_numbers[-10:]

['LO/604371/2013',
 'KS/356120/2014',
 'LO/741738/2011',
 'KA/737794/2018',
 'SP/564298/2011',
 'KA/737818/2012',
 'MA/513454/2009',
 'IT/662447/2017',
 'IT/327794/2015',
 'SP/258305/2012']

Również ok, przynajmniej na pierwszy rzut oka... (Praca domowa będzie o tym!)

### Czyszczenie `employment_start_date`

In [98]:
people = PersonalDataCleaner().clean(get_csv_lines())

[person['employment_start_date'] for person in people][:25]

['2014-09-06T00:00:00',
 '2010-04-20',
 '2014-04-09',
 '2010-09-17',
 '2017-09-10T00:00:00',
 '2012-05-09',
 '2018-03-29',
 '2017-12-19',
 '2012-11-25',
 '2009/11/21',
 '2010-06-09',
 '2017/12/19',
 '2013/04/24',
 '2015-05-14T00:00:00',
 '2009-06-24',
 '2011-05-25',
 '2011-07-14',
 '2018-07-07',
 '2012-08-17',
 '2017-12-19',
 '2014-07-18',
 '2017-01-03',
 '2013-09-21',
 '2018/05/18',
 '2017-09-10T00:00:00']

Nie jest źle, mamy tylko trzy formaty:

`YYYY-MM-DD`

`YYYY/MM/DD`

`YYYY-MM-DD{T}HH:MM:SS`

Można to ogarnąć na różne sposoby, ale zróbmy to na piechotę:

1. Wyciągniemy rok, miesiąc i dzień ze stringów.

2. Stworzymy obiekty typu `date`.

Typ **`date`**: https://docs.python.org/3.7/library/datetime.html#date-objects.

In [99]:
from datetime import date

my_date = date(year=2013, month=11, day=15)

In [100]:
my_date

datetime.date(2013, 11, 15)

Użyjmy metody **`partition`**: https://docs.python.org/3/library/stdtypes.html#str.partition.

In [101]:
'ble ble AAA bla bla AAA'.partition('BBB')

('ble ble AAA bla bla AAA', '', '')

In [102]:
def normalize_date(date_string):
    if date_string is None:
        return None

    # YYYY-MM-DD{T}HH:MM:SS --> YYYY-MM-DD
    date_string = date_string.partition('T')[0]
    
    # case 2: YYYY/MM/DD --> YYYY-MM-DD
    date_string = date_string.replace('/', '-')
    
    # convert to `date` object (exception on wrong format)
    year, month, day = date_string.split('-')
    
    date_obj = date(year=int(year), month=int(month), day=int(day))
    
    return date_obj

In [103]:
print(normalize_date('2017-12-07'))
print(normalize_date('2017/12/07'))
print(normalize_date('2017-12-07T00:00:00'))
print(normalize_date(None))

2017-12-07
2017-12-07
2017-12-07
None


### UWAGA!!!

1. Nie uwzględniam innych zapisów, np. `DD-MM-YYYY` albo `MM-DD-YYYY`.

2. Ignoruję całe wielkie zagadnienie stref czasowych. Generalnie zasada jest taka, że **daty przesyłamy i zapisujemy w strefie UTC, bo inaczej popadniemy w duuuuuże problemy**.

Jest taka biblioteka `dateutil` (https://dateutil.readthedocs.io/en/stable), która m.in. posiada funkcję parsowania dat zapisanych w różnych formatach. Takich narzędzi trzeba używać bardzo ostrożnie jednak, bo łatwo się przejechać...

### Czyszczenie `monthly_salary`

In [104]:
people = PersonalDataCleaner().clean(get_csv_lines())

[person['monthly_salary'] for person in people][:10]

['2436.9',
 '3407.4',
 '4676.760000000001',
 '5533.968000000002',
 '4 731,210000000001',
 '5306.895000000001',
 '3746.6000000000004',
 '5184.0',
 '5 055,299999999999',
 '4195.2735']

Mamy przynajmniej dwa formatowania:

- `1234.5`

- `1 234,5`

A dodatkowo czasem mamy problemy z liczbą miejsc po przecinku.

Ogarnijmy to po kolei:

1. zamiana stringów na floaty

2. ogarnięcie zaokrągleń

Na szczęście nie mamy liczb z zarówno `.` jak i `,`, więc czyszczenie jest proste:

In [105]:
def monetary_string_to_float(string):
    string = string.replace(" ", "")
    string = string.replace(',', '.')
    return float(string)

In [106]:
print(monetary_string_to_float('6414.1'))
print(monetary_string_to_float('6414.0999999999999'))
print(monetary_string_to_float('6 414,1'))
print(monetary_string_to_float('6 414,0999999999999'))

6414.1
6414.099999999999
6414.1
6414.099999999999


Teraz ogarnijmy zaokrąglenia, by to jakoś wyglądało sensownie...

Float się nie nadaje, potrzebujemy bardziej wyspecjalizowanego typu danych: **`Decimal`**. Jest to typ danych idealny do zastosowań finansowych.

https://docs.python.org/3/library/decimal.html

In [107]:
from decimal import Decimal

Decimal('1.01')

Decimal('1.01')

In [108]:
from decimal import Decimal

Decimal('1.0999999999999')

Decimal('1.0999999999999')

Metoda **`quantize`** (kwantyfikacja) ucina i zaokrągla: https://docs.python.org/3.7/library/decimal.html#decimal.Decimal.quantize.

In [109]:
d1 = Decimal('1.0999999999999').quantize(Decimal('0.01'))
d2 = Decimal('1.99').quantize(Decimal('0.01'))

Można je tworzyć od razu ze stringów, więc napiszmy drugą wersję naszej funkcji:

In [110]:
from decimal import Decimal

def monetary_string_to_decimal(string):
    if string is None:
        return None
    string = string.replace(' ', '')
    string = string.replace(',', '.')
    return Decimal(string).quantize(Decimal('0.01'))

In [111]:
print(monetary_string_to_decimal('6414.1'))
print(monetary_string_to_decimal('6414.0999999999999'))
print(monetary_string_to_decimal('6 414,1'))
print(monetary_string_to_decimal('6 414,0999999999999'))

6414.10
6414.10
6414.10
6414.10


Elegancko!

### Wrzucenie wszystkich funkcji czyszczących do klasy `PersonalDataCleaner`

Wrzućmy wszystko do cleaner'a naszego!

In [112]:
from collections import OrderedDict
from datetime import date
from decimal import Decimal


class PersonalDataCleaner:
    """
    Cleaner of personal data. 
    
    Usage:
      Pass an iterable of dicts to `clean` method, which 
      will yield dicts with cleaned data.
    """
    
    STRINGS_REPRESENTING_TRUE = {'1', 'prawda', 'tak', 'ma', 'true'}
    STRINGS_REPRESENTING_FALSE = {'0', 'fałsz', 'nie', 'nie ma', 'false'}
    
    def __init__(self):
        map_string_to_true = {
            k: True
            for k in self.STRINGS_REPRESENTING_TRUE
        }
        map_string_to_false = {
            k: False
            for k in self.STRINGS_REPRESENTING_FALSE
        }
        self.map_string_to_bool = {
            **map_string_to_true,
            **map_string_to_false,
        }

    def clean(self, rows):
        for row in rows:
            # czyszczenie wspólne dla wszystkich kolumn
            clean_row = self._clean_all_items(row)

            # czyszczenie wyspecjalizowane
            clean_row['first_name'] = (
                self._clean_name(clean_row['first_name'])
            )
            clean_row['last_name'] = (
                self._clean_name(clean_row['last_name'])
            )
            clean_row['id_number'] = (
                self._clean_id_number(clean_row['id_number'])
            )
            clean_row['employment_start_date'] = (
                self._clean_date(clean_row['employment_start_date'])
            )
            clean_row['monthly_salary'] = (
                self._clean_monetary_value(clean_row['monthly_salary'])
            )
            clean_row['department'] = (
                self._clean_department(clean_row['department'])
            )
            clean_row['multisport'] = (
                self._clean_multisport(clean_row['multisport'])
            )
    
            yield clean_row
            
    def _clean_all_items(self, row):
        clean_row = OrderedDict()
        for key, value in row.items():
            clean_value = value.strip()
            clean_value = clean_value or None
            clean_row[key] = clean_value
        return clean_row

    def _clean_name(self, name):
        if name is None:
            return None
        return name.title()

    def _clean_id_number(self, id_number):
        return id_number

    def _clean_date(self, date_string):
        if date_string is None:
            return None

        # YYYY-MM-DD{T}HH:MM:SS --> YYYY-MM-DD
        date_string = date_string.partition('T')[0]
    
        # YYYY/MM/DD --> YYYY-MM-DD
        date_string = date_string.replace('/', '-')
    
        # convert to `date` object (exception on wrong format)
        year, month, day = date_string.split('-')
        date_obj = date(year=int(year), month=int(month), day=int(day))
    
        return date_obj

    def _clean_monetary_value(self, amount):
        if amount is None:
            return None
        amount = amount.replace(' ', '')
        amount = amount.replace(',', '.')
        return Decimal(amount).quantize(Decimal('0.01'))

    def _clean_department(self, department):
        return department

    def _clean_multisport(self, something):
        if something is None:
            return None
        something = something.lower()
        return self.map_string_to_bool[something]

In [113]:
people = PersonalDataCleaner().clean(get_csv_lines())

In [114]:
list(people)[:3]

[OrderedDict([('first_name', 'Urszula'),
              ('last_name', 'Szewczyk'),
              ('id_number', 'CZ/594959/2014'),
              ('employment_start_date', datetime.date(2014, 9, 6)),
              ('monthly_salary', Decimal('2436.90')),
              ('department', 'CZYSTOŚĆ'),
              ('multisport', False)]),
 OrderedDict([('first_name', 'Stanisława'),
              ('last_name', 'Kowalska'),
              ('id_number', 'LO/374979/2010'),
              ('employment_start_date', datetime.date(2010, 4, 20)),
              ('monthly_salary', Decimal('3407.40')),
              ('department', 'LOGISTYKA'),
              ('multisport', False)]),
 OrderedDict([('first_name', 'Ilona'),
              ('last_name', 'Pawlak'),
              ('id_number', 'SP/598919/2014'),
              ('employment_start_date', datetime.date(2014, 4, 9)),
              ('monthly_salary', Decimal('4676.76')),
              ('department', 'SPRZEDAŻ'),
              ('multisport', True)])]

### Zapisanie danych do pliku CSV

In [115]:
import csv

def save_csv(data):
    first_row = next(data)
    field_names = first_row.keys()
    with open(out_filename, 'w', newline='') as csvfile:
        csv_writer = csv.DictWriter(
            csvfile,
            delimiter=',',
            fieldnames=field_names,
        )
        csv_writer.writeheader()
        csv_writer.writerow(first_row)
        for row in data:
            csv_writer.writerow(row)

In [116]:
people = PersonalDataCleaner().clean(get_csv_lines())
save_csv(people)

# 2. OBRÓBKA DANYCH

Teraz mamy dane przygotowane do obrabiania. Pytania, na które mamy odpowiedzieć:

1. Wyznaczyć średnią i medianę wynagrodzeń.

2. Wyznaczyć średnią i medianę wynagrodzeń w poszczególnych działach firmy.

3. Sprawdzić, czy posiadanie karty multisport związane jest z wysokością zarobków.

4. Określic liczbę osób pracujących w poszczególnych działach firmy.

5. Znaleźć osobę zarabiającą najwięcej oraz osobę zarabiającą najmniej.

6. Sprawdzić kiedy zatrudniony został pierwszy pracownik działu kadry.

## 2.1. Średnia i mediana wynagrodzeń

In [117]:
people = list(PersonalDataCleaner().clean(get_csv_lines()))

salaries = [
    person['monthly_salary']
    for person in people 
    if person['monthly_salary'] is not None
]

In [118]:
salaries[:10]

[Decimal('2436.90'),
 Decimal('3407.40'),
 Decimal('4676.76'),
 Decimal('5533.97'),
 Decimal('4731.21'),
 Decimal('5306.90'),
 Decimal('3746.60'),
 Decimal('5184.00'),
 Decimal('5055.30'),
 Decimal('4195.27')]

Możemy liczyć na piechotę, ale w Pythonie są już odpowiednie funkcje na to.

In [119]:
from statistics import mean

mean(salaries)

Decimal('5276.695185909980430528375734')

Funkcja **`mean`**: https://docs.python.org/3/library/statistics.html#statistics.mean.

In [120]:
from decimal import Decimal

mean(salaries).quantize(Decimal('0.01'))

Decimal('5276.70')

In [121]:
from statistics import median

median(salaries).quantize(Decimal('0.01'))

Decimal('4300.56')

Funkcja **`median`**: https://docs.python.org/3/library/statistics.html#statistics.median

## 2.2. Średnia i mediana wynagrodzeń w poszczególnych działach firmy

In [122]:
from collections import defaultdict

dep_to_salaries = defaultdict(list)
for person in people:
    salary = person['monthly_salary']
    department = person['department']
    if department is not None:
        dep_to_salaries[department].append(salary)

Typ **`defaultdict`**: https://docs.python.org/3.7/library/collections.html#collections.defaultdict.

In [123]:
dep_to_salaries['SPRZEDAŻ'][:10]

[Decimal('4676.76'),
 Decimal('4731.21'),
 Decimal('5306.90'),
 Decimal('5055.30'),
 Decimal('3258.75'),
 Decimal('3387.00'),
 Decimal('5040.00'),
 Decimal('3269.70'),
 Decimal('3385.50'),
 Decimal('3483.81')]

In [124]:
for department, salary_list in dep_to_salaries.items():
    salary_mean = mean(salary_list).quantize(Decimal('0.01'))
    salary_median = median(salary_list).quantize(Decimal('0.01'))
    
    print(f'{department}')
    print(f'\t mean   = {salary_mean:10.2f}')
    print(f'\t median = {salary_median:10.2f}')

CZYSTOŚĆ
	 mean   =    2842.85
	 median =    2800.50
LOGISTYKA
	 mean   =    3881.02
	 median =    3862.12
SPRZEDAŻ
	 mean   =    4334.24
	 median =    4332.22
MARKETING
	 mean   =    4591.66
	 median =    4598.40
OCHRONA
	 mean   =    3206.21
	 median =    3212.60
KSIĘGOWOŚĆ
	 mean   =    5732.21
	 median =    5645.86
KADRY
	 mean   =    4418.94
	 median =    4358.83
ZARZĄD
	 mean   =   90787.42
	 median =   64721.38
IT
	 mean   =    7347.72
	 median =    7395.00


Co to to takie dziwne **`:10.2f`** w f-stringu? Zob. https://docs.python.org/3/library/string.html#format-specification-mini-language.

## 2.3. Czy posiadanie karty multisport związane jest z wysokością zarobków

In [125]:
multisport_to_salaries = defaultdict(list)

for person in people:
    salary = person['monthly_salary']
    multisport = person['multisport']
    if multisport is not None:
        multisport_to_salaries[multisport].append(salary)

In [126]:
print('Multisport')

for multisport, salaries in multisport_to_salaries.items():
    salary_mean = mean(salaries).quantize(Decimal('0.01'))
    salary_median = median(salaries).quantize(Decimal('0.01'))
    
    print(f'\t {multisport}')
    print(f'\t\t mean   = {salary_mean:10.2f}')
    print(f'\t\t median = {salary_median:10.2f}')

Multisport
	 False
		 mean   =    5290.12
		 median =    4229.11
	 True
		 mean   =    5096.26
		 median =    4495.10


## 2.4. Liczba osób pracujących w poszczególnych działach firmy

In [127]:
from collections import Counter

counter = Counter()

counter['a'] += 1
counter['a'] += 1
counter['a'] += 1

counter['b'] += 55

counter

Counter({'a': 3, 'b': 55})

Typ **`Counter`**: https://docs.python.org/3.7/library/collections.html#collections.Counter.

In [128]:
department_to_employee_count = Counter()

for person in people:
    department = person['department']
    if department:
        department_to_employee_count[department] += 1

sorted(
    department_to_employee_count.items(),
    key=lambda x: x[1],
    reverse=True,
)

[('SPRZEDAŻ', 334),
 ('MARKETING', 253),
 ('LOGISTYKA', 240),
 ('KSIĘGOWOŚĆ', 199),
 ('KADRY', 185),
 ('CZYSTOŚĆ', 160),
 ('OCHRONA', 70),
 ('IT', 39),
 ('ZARZĄD', 16)]

Funkcja **`sorted`** z parametrami `key` oraz `reverse`: https://docs.python.org/3/library/functions.html#sorted.

## 2.5. Kto zarabia najwięcej a kto najmniej

In [129]:
salaries = [
    person['monthly_salary']
    for person in people 
    if person['monthly_salary'] is not None
]

In [130]:
max(salaries)

Decimal('436050.00')

Funkcja **`max`**: https://docs.python.org/3/library/functions.html#max.

:O ... chyba CEO znaleźliśmy ;)

In [131]:
min(salaries)

Decimal('2126.10')

Funkcja **`min`**: https://docs.python.org/3/library/functions.html#min.

OK, ale kto jest kim...

In [132]:
max(people, key=lambda person: person['monthly_salary'])

OrderedDict([('first_name', 'Renata'),
             ('last_name', 'Pawlak'),
             ('id_number', 'ZA/760256/2015'),
             ('employment_start_date', datetime.date(2015, 5, 14)),
             ('monthly_salary', Decimal('436050.00')),
             ('department', 'ZARZĄD'),
             ('multisport', False)])

In [133]:
min(people, key=lambda person: person['monthly_salary'])

OrderedDict([('first_name', 'Bogumiła'),
             ('last_name', 'Nowicka'),
             ('id_number', 'CZ/471338/2013'),
             ('employment_start_date', datetime.date(2013, 9, 21)),
             ('monthly_salary', Decimal('2126.10')),
             ('department', 'CZYSTOŚĆ'),
             ('multisport', False)])

## 2.6. Kiedy zatrudniony został pierwszy pracownik działu kadry

In [134]:
hr_people = [
    person
    for person in people
    if person['department'] == 'KADRY'
    if person['employment_start_date'] is not None
]

In [135]:
hr_empl = min(hr_people, 
              key=lambda person: person['employment_start_date'])
hr_empl['employment_start_date']
hr_empl

OrderedDict([('first_name', 'Józefa'),
             ('last_name', 'Adamczyk'),
             ('id_number', 'KA/165959/2009'),
             ('employment_start_date', datetime.date(2009, 8, 13)),
             ('monthly_salary', Decimal('4082.08')),
             ('department', 'KADRY'),
             ('multisport', False)])

# 3. WIZUALIZACJA DANYCH

Użyjemy biblioteki **Bokeh**: https://bokeh.pydata.org/.

In [136]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

## Średnie zarobki w poszczególnych działach

In [137]:
from collections import defaultdict
from statistics import mean


department_to_salaries = defaultdict(list)

for person in people:
    salary = person['monthly_salary']
    department = person['department']
    if department is not None:
        department_to_salaries[department].append(salary)
        
department_to_salary_mean = tuple(
    (department, mean(salaries).quantize(Decimal('0.01')))
    for department, salaries in department_to_salaries.items()
    if department not in {None} #, 'ZARZĄD'}
)

department_to_salary_mean = sorted(department_to_salary_mean, key=lambda t: t[1])

In [138]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral9, Colorblind8
from bokeh.models import NumeralTickFormatter, DatetimeTickFormatter

In [139]:
def draw_mean_salary(departments, means, palette):
    source = ColumnDataSource(data=dict(
        departments=departments, means=means, color=palette,
    ))

    p = figure(
        x_range=departments,
        y_range=(0, int(max(means))*1.1), 
        plot_height=250,
        plot_width=700,
        title="Średnie wynagrodzenie w poszczególnych działach",
    )
    p.vbar(
        x='departments',
        top='means',
        width=0.5,
        color='color',
        legend=False,
        source=source,
    )

    p.xgrid.grid_line_color = None
    p.yaxis[0].formatter = NumeralTickFormatter(format="0.00")

    show(p)

In [140]:
departments = [item[0] for item in department_to_salary_mean]
means = [item[1] for item in department_to_salary_mean]
draw_mean_salary(departments, means, Spectral9)

ok... zobaczmy bez zarządu:

In [141]:
departments = [item[0] for item in department_to_salary_mean][:-1]
means = [item[1] for item in department_to_salary_mean][:-1]

draw_mean_salary(departments, means, Colorblind8)

## Zarobki w dziale IT a data zatrudnienia

Zobaczmy, czy starci stażem pracownicy zarabiają więcej.

In [142]:
people_with_all_needed_data = tuple(
    person
    for person in people
    if person['monthly_salary'] is not None
    if person['employment_start_date'] is not None
)

it_monthly_salaries = tuple(
    person['monthly_salary']
    for person in people_with_all_needed_data
    if person['department'] == 'IT'
)

it_emlp_start_dates = tuple(
    person['employment_start_date']
    for person in people_with_all_needed_data
    if person['department'] == 'IT'
)

In [143]:
p = figure(plot_width=700, plot_height=400, y_axis_type="datetime")

p.circle(
    it_emlp_start_dates,
    it_monthly_salaries,
    size=15,
    line_color="navy",
    fill_color="blue",
    fill_alpha=1,
)

p.yaxis[0].formatter = NumeralTickFormatter(format="0.00")
p.xaxis[0].formatter = DatetimeTickFormatter()

show(p)

Wygląda na to, że nie koreluje.

**`¯\_(ツ)_/¯`**